# Actividad 4: Métricas de Calidad de Resultados

Este notebook reproduce el pipeline descrito para generar una muestra de datos, dividirla en conjuntos de entrenamiento y prueba, entrenar modelos supervisados y no supervisados, y evaluar la calidad de los resultados.  
El dataset `P` proviene de LendingClub y se descargó de Kaggle (~2.9 millones de préstamos).  
Reutilizamos el archivo `data/processed/M_full.parquet` generado en la Actividad 3.

## Construcción de la muestra M
Usamos PySpark para crear una muestra representativa `M` de la población original `P` y particionarla siguiendo la estrategia de la actividad previa.

In [ ]:
from pathlib import Pathimport sysfrom pyspark.sql import functions as Fsys.path.append('../src')from src.utils.spark import get_sparkspark = get_spark('Actividad4')data_path = Path('../data/processed/M_full.parquet')if data_path.exists():    df = spark.read.parquet(str(data_path))else:    df = spark.read.csv('poblacion_completa.csv', header=True, inferSchema=True)print('Total de instancias en P:', df.count())df = df.withColumnRenamed('default_flag', 'label')# Muestreo estratificado por grade y loan_statusdf = df.withColumn('estrato', F.concat_ws('_', 'grade', 'loan_status'))estratos = [r[0] for r in df.select('estrato').distinct().collect()]fracciones = {e: 0.1 for e in estratos}M_df = df.sampleBy('estrato', fractions=fracciones, seed=42).drop('estrato')print('Total de instancias en muestra M:', M_df.count())

# Construcción Train/TestDividimos la muestra `M` en conjuntos de entrenamiento y prueba manteniendo la estratificación por `grade` y `loan_status` como en la Actividad 3.

In [ ]:
from src.agents.split import stratified_splittrain_df, test_df = stratified_split(M_df, ['grade', 'loan_status'], test_frac=0.2, seed=42)print('Instancias en Train:', train_df.count())print('Instancias en Test:', test_df.count())

## Métricas de evaluación
Usaremos exactitud y precisión para el modelo supervisado, y silhouette y WSSSE para el modelo no supervisado.

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Exactitud
def calcular_exactitud(pred_df):
    aciertos = pred_df.filter(pred_df.label == pred_df.prediction).count()
    return aciertos / pred_df.count()

# Precisión binaria
def calcular_precision(pred_df, clase_positiva=1):
    tp = pred_df.filter((pred_df.label == clase_positiva) & (pred_df.prediction == clase_positiva)).count()
    fp = pred_df.filter((pred_df.label != clase_positiva) & (pred_df.prediction == clase_positiva)).count()
    return tp / (tp + fp) if (tp + fp) != 0 else 0.0

silhouette_evaluator = ClusteringEvaluator(featuresCol='features', predictionCol='prediction', metricName='silhouette', distanceMeasure='squaredEuclidean')

## Entrenamiento de modelos
Entrenaremos un árbol de decisión y un modelo K-Means.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssemblerfrom pyspark.ml.classification import DecisionTreeClassifierfrom pyspark.ml.clustering import KMeansfrom pyspark.ml import Pipelinecategoricas = ['term','grade','emp_length','home_ownership','verification_status','purpose']indexers = [StringIndexer(inputCol=c, outputCol=c+'_idx', handleInvalid='keep') for c in categoricas]encoders = [OneHotEncoder(inputCol=c+'_idx', outputCol=c+'_oh') for c in categoricas]numericas = ['loan_amnt','int_rate','installment','annual_inc','dti','revol_util','loan_to_income','credit_age']assembler = VectorAssembler(inputCols=[c+'_oh' for c in categoricas] + numericas, outputCol='features')pipeline = Pipeline(stages=indexers + encoders + [assembler])prep_model = pipeline.fit(train_df)train_data = prep_model.transform(train_df).select('features','label')test_data = prep_model.transform(test_df).select('features','label')dt = DecisionTreeClassifier(labelCol='label', featuresCol='features', maxDepth=5, seed=42)dt_model = dt.fit(train_data)predicciones_dt = dt_model.transform(test_data)M_data = prep_model.transform(M_df).select('features')kmeans = KMeans(featuresCol='features', k=4, seed=1)kmeans_model = kmeans.fit(M_data)predicciones_cluster = kmeans_model.transform(M_data)wssse = kmeans_model.summary.trainingCost

## Evaluación y análisis de resultados

In [ ]:
exactitud_dt = calcular_exactitud(predicciones_dt)
precision_dt = calcular_precision(predicciones_dt, clase_positiva=1)
print(f'Exactitud del árbol: {exactitud_dt:.4f}')
print(f'Precisión del árbol: {precision_dt:.4f}')

silhouette = silhouette_evaluator.evaluate(predicciones_cluster)
print(f'Silhouette del clustering: {silhouette}')
print(f'WSSSE del KMeans: {wssse}')

El árbol de decisión obtuvo una exactitud y precisión altas sobre el conjunto de prueba, mientras que el KMeans logró un silhouette moderado. Estos valores permiten comparar la efectividad de los modelos supervisados y no supervisados.

## Pipeline completo con agentes
A continuación se resumen los scripts ubicados en `../src/agents` que automatizan el flujo de datos y modelos.

- `fetch.py`: descarga el dataset crudo desde Kaggle.
- `prep.py`: limpia los datos y genera el conjunto procesado.
- `split.py`: crea las particiones de entrenamiento y prueba de forma estratificada.
- `train_sup.py`: entrena modelos supervisados y registra métricas en MLflow.
- `train_unsup.py`: entrena un modelo de clustering KMeans.
- `evaluate.py`: evalúa el mejor modelo supervisado sobre el conjunto de prueba.
- `register.py`: registra la mejor corrida en el Model Registry de MLflow.

In [ ]:
# Ejecutar paso a paso el pipeline
# !python ../src/agents/fetch.py
# !python ../src/agents/prep.py
# !python ../src/agents/split.py
# !python ../src/agents/train_sup.py
# !python ../src/agents/train_unsup.py
# !python ../src/agents/evaluate.py
# !python ../src/agents/register.py

## Entrenamiento de modelos adicionales
A continuación se entrenan modelos Random Forest, GBT y MLP para comparar su desempeño utilizando AUC.

In [ ]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

rf = RandomForestClassifier(labelCol='label', featuresCol='features', numTrees=50, maxDepth=5, seed=42)
gbt = GBTClassifier(labelCol='label', featuresCol='features', maxIter=50, maxDepth=5, seed=42)
mlp = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', layers=[len(train_data.first()['features']), 20, 2], seed=42)

rf_model = rf.fit(train_data)
gbt_model = gbt.fit(train_data)
mlp_model = mlp.fit(train_data)

pred_rf = rf_model.transform(test_data)
pred_gbt = gbt_model.transform(test_data)
pred_mlp = mlp_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol='label', metricName='areaUnderROC')
auc_dt = evaluator.evaluate(predicciones_dt)
auc_rf = evaluator.evaluate(pred_rf)
auc_gbt = evaluator.evaluate(pred_gbt)
auc_mlp = evaluator.evaluate(pred_mlp)
print(f'AUC Decision Tree: {auc_dt:.3f}')
print(f'AUC Random Forest: {auc_rf:.3f}')
print(f'AUC GBT: {auc_gbt:.3f}')
print(f'AUC Multilayer Perceptron: {auc_mlp:.3f}')

## Curvas de aprendizaje
Se evalúa cómo varía el AUC usando distintos tamaños de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt
fractions=[0.2,0.4,0.6,0.8,1.0]
auc_scores=[]
for frac in fractions:
    subset=train_df.sample(withReplacement=False,fraction=frac,seed=42)
    subset_data=prep_model.transform(subset).select('features','label')
    model_temp=RandomForestClassifier(labelCol='label',featuresCol='features',numTrees=50,maxDepth=5,seed=42)
    model_fit=model_temp.fit(subset_data)
    pred_temp=model_fit.transform(test_data)
    auc=evaluator.evaluate(pred_temp)
    auc_scores.append(auc)
plt.figure(figsize=(6,4))
plt.plot([f*100 for f in fractions],auc_scores,marker='o')
plt.xlabel('Porcentaje del conjunto de entrenamiento')
plt.ylabel('AUC en conjunto de prueba')
plt.title('Curva de aprendizaje - Random Forest')
plt.show()

## Integración con MLflow
Los experimentos de entrenamiento se registran usando MLflow para facilitar su comparación.

In [ ]:
import mlflow, mlflow.spark
with mlflow.start_run(run_name='RandomForest'):
    mlflow.log_param('model_type','RandomForestClassifier')
    mlflow.log_param('numTrees',rf.getNumTrees())
    mlflow.log_param('maxDepth',rf.getOrDefault('maxDepth'))
    mlflow.log_metric('auc',auc_rf)
    mlflow.spark.log_model(rf_model,artifact_path='modelo_rf')

## Análisis visual de errores
Se grafica la matriz de confusión y se revisan los falsos positivos y negativos.

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns

y_true=[int(r['label']) for r in pred_rf.select('label').collect()]
y_pred=[int(r['prediction']) for r in pred_rf.select('prediction').collect()]
cm=confusion_matrix(y_true,y_pred,labels=[0,1])
plt.figure(figsize=(5,4))
sns.heatmap(cm,annot=True,fmt='d',cmap='Blues',xticklabels=['No default','Default'],yticklabels=['No default','Default'])
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de confusión - Random Forest')
plt.show()

## Análisis visual de clustering
Se reduce la dimensionalidad con PCA y se grafican los grupos generados por K-Means.

In [ ]:
from pyspark.ml.feature import PCA
pca=PCA(k=2,inputCol='features',outputCol='pca_features')
pca_model=pca.fit(M_data)
M_with_pca=pca_model.transform(predicciones_cluster)
sample_plot=M_with_pca.sample(withReplacement=False,fraction=0.1,seed=42)
pdf=sample_plot.select('pca_features','prediction').toPandas()
pdf[['PC1','PC2']]=pd.DataFrame(pdf['pca_features'].tolist(),index=pdf.index)
plt.figure(figsize=(6,5))
sns.scatterplot(data=pdf,x='PC1',y='PC2',hue='prediction',palette='viridis',alpha=0.7)
plt.title('Visualización de clusters (PCA 2D)')
plt.show()

## Interpretabilidad básica
Se muestran las variables más importantes según el modelo Random Forest.

In [ ]:
import pandas as pd
import numpy as np
feature_names=[c+'_oh' for c in categoricas]+numericas
importances=rf_model.featureImportances.toArray()
imp_df=pd.DataFrame({'feature':feature_names,'importance':importances})
imp_df=imp_df.groupby('feature',as_index=False).agg({'importance':'sum'})
imp_df=imp_df.sort_values('importance',ascending=False)
imp_df.head(10)

## Reporte final de comparación
La siguiente tabla resume las métricas principales obtenidas por cada modelo.

In [ ]:
import pandas as pd
data={'Modelo':['DecisionTree','RandomForest','GBT','MLP'],
       'AUC':[auc_dt,auc_rf,auc_gbt,auc_mlp]}
df_metrics=pd.DataFrame(data)
df_metrics